<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/huggingface/hf_nlp_02_use_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用transformer

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

运行结果：

```
[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]
```
pipeline将三个步骤组合在一起：**预处理**、**向模型传递输入**以及**后处理**：

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg" alt="pipeline处理步骤" style="width: 500px; height: 300px;"/>





### pipeline内部原理

#### 使用tokenizer进行预处理
tokenizer作用将文本输入转换为模型可以理解的数字。主要处理内容：
- 将输入拆分为单词、子单词或符号（如标点符号），这些单词、子单词或符号（如标点符号）被称为tokens
- 将每个tokens映射到一个整数
- 添加可能对模型有用的额外输入

预处理都需要以与预训练模型时完全相同的方式完成。我们可以使用`AutoTokenizer.from_pretrained(checkpoint)`获取与模型的标记生成器关联的数据并缓存它。

查看`sentiment-analysis`模型 `checkpoint`（默认为`distilbert-base-uncased-finetuned-sst-2-english `）标记数据，如下：


In [4]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
# tensorflow
# inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


#### 向模型传递输入
- 使用`AutoModel.from_pretrained(checkpoint)`实例化一个模型
- 将预处理后的数据传递给实例化后的模型

通过上面两步处理，将得到模型的高维向量：

**Batch size**：一次处理的序列数(（即预处理结果中input_ids大小）)
**Sequence length**：序列的数字表示的长度（即预处理结果中input_ids元素长度）
**Hidden size**：每个模型输入的向量维度

In [6]:
from transformers import AutoModel
#tensorflow 使用 TFAutoModel
# from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs)
# shape 函数返回一个表示张量维度的元组。对于三维数组，返回形如 (n, m, p) 的元组
print(outputs.last_hidden_state.shape)

BaseModelOutput(last_hidden_state=tensor([[[-0.1798,  0.2333,  0.6321,  ..., -0.3017,  0.5008,  0.1481],
         [ 0.2758,  0.6497,  0.3200,  ..., -0.0760,  0.5136,  0.1329],
         [ 0.9046,  0.0985,  0.2950,  ...,  0.3352, -0.1407, -0.6464],
         ...,
         [ 0.1466,  0.5661,  0.3235,  ..., -0.3376,  0.5100, -0.0561],
         [ 0.7500,  0.0487,  0.1738,  ...,  0.4684,  0.0030, -0.6084],
         [ 0.0519,  0.3729,  0.5223,  ...,  0.3584,  0.6500, -0.3883]],

        [[-0.2937,  0.7283, -0.1497,  ..., -0.1187, -1.0227, -0.0422],
         [-0.2206,  0.9384, -0.0951,  ..., -0.3643, -0.6605,  0.2407],
         [-0.1536,  0.8988, -0.0728,  ..., -0.2189, -0.8528,  0.0710],
         ...,
         [-0.3017,  0.9002, -0.0200,  ..., -0.1082, -0.8412, -0.0861],
         [-0.3338,  0.9674, -0.0729,  ..., -0.1952, -0.8181, -0.0634],
         [-0.3454,  0.8824, -0.0426,  ..., -0.0993, -0.8329, -0.1065]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)
t

##### model传递输入内部机制

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/transformer_and_head.svg"/>

在这个图中。模型有`embeddings layer`和 `subsequent layers`。
- `embeddings layer`将标记化输入中的每个输入 ID 转换为表示关联标记的向量。
- `subsequent layers`使用注意力机制操纵这些向量来产生句子的最终表示。

`model heads`将隐藏状态的高维向量作为输入，并将它们投影到不同的维度上。（Transformer模型的输出直接发送到模型头进行处理）


Transformers 中可用的架构（每种架构都是围绕解决特定任务而设计的）：
- *Model (retrieve the hidden states)
- *ForCausalLM
- *ForMaskedLM
- *ForMultipleChoice
- *ForQuestionAnswering
- *ForSequenceClassification
- *ForSequenceClassification
- and others

In [7]:
from transformers import AutoModelForSequenceClassification

#tensorflow 使用 TFAutoModelForSequenceClassification
# from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)
print(outputs)
print(outputs.logits.shape)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
torch.Size([2, 2])


#### 后置处理输出
上面模型预测第一个句子的 [-1.5607, 1.6123] 和第二个句子的 [ 4.1692, -3.3464]。这些不是概率，要转换为概率，需要经过 SoftMax 层

In [10]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

# tensorflow框架
# import tensorflow as tf

# predictions = tf.math.softmax(outputs.logits, axis=-1)
# print(predictions)

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)


可以看到模型为第一个句子预测了 [0.0402, 0.9598] ，为第二个句子预测了 [0.9995, 0.0005] 。这些是可识别的概率分数。

In [11]:
# 获取与每个位置相对应的标签
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}

### 模型的创建和使用


#### 创建Transformer

##### 使用默认方式创建：

从默认配置创建模型会使用随机值对其进行初始化。这种方式模型可以使用，但会输出乱码；它需要先经过训练。为了避免不必要的重复工作，可以共享和重用已经训练过的模型。

In [13]:
from transformers import BertConfig, TFBertModel

# Building the config
config = BertConfig()

# Building the model from the config
model = TFBertModel(config)

print(config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



##### 使用训练好的的模型
可以使用 `from_pretrained()` 方法加载已经训练好的 Transformer 模型。
这种方式不需要使用 `BertConfig` ，而是通过 `bert-base-cased `标识符加载预训练模型，使用模型检查点的所有权重进行初始化。

用于加载模型的标识符可以是模型中心上任何模型的标识符，只要它与 BERT 架构兼容即可。
我们可以使用 https://huggingface.co/models?other=bert 来查看哪些模型与BERT 架构兼容。





In [3]:
from transformers import TFBertModel

model = TFBertModel.from_pretrained("bert-base-cased")

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

##### 保存模型

使用 save_pretrained() 方法保存模型。这会将`config.json` `tf_model.h5`两个文件保存到您的磁盘上。
> 如果使用的是pytorch生成的是config.json、 pytorch_model.bin两个文件

- config.json ：该文件还包含一些元数据，例如检查点的起源位置以及上次保存检查点时使用的  Transformers 版本。
- tf_model.h5 ： 被称为状态字典；它包含模型的所有权重。


In [15]:
model.save_pretrained("directory_on_my_computer")

#### 使用Transformer 模型进行推理



In [1]:
sequences = ["Hello!", "Cool.", "Nice!"]
# tokenizer 转换为input ID
encoded_sequences = [
    [101, 7592, 999, 102],
    [101, 4658, 1012, 102],
    [101, 3835, 999, 102],
]

In [5]:
import tensorflow as tf

model_inputs = tf.constant(encoded_sequences)

# import torch

# model_inputs = torch.tensor(encoded_sequences)

output = model(model_inputs)
print(output)

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(3, 4, 768), dtype=float32, numpy=
array([[[ 4.4495744e-01,  4.8276284e-01,  2.7797198e-01, ...,
         -5.4032274e-02,  3.9393425e-01, -9.4769992e-02],
        [ 2.4942899e-01, -4.4092962e-01,  8.1772339e-01, ...,
         -3.1916568e-01,  2.2992219e-01, -4.1171715e-02],
        [ 1.3667546e-01,  2.2517854e-01,  1.4501992e-01, ...,
         -4.6914592e-02,  2.8224233e-01,  7.5565845e-02],
        [ 1.1788858e+00,  1.6738570e-01, -1.8187097e-01, ...,
          2.4671350e-01,  1.0440764e+00, -6.1971312e-03]],

       [[ 3.6435828e-01,  3.2463297e-02,  2.0257683e-01, ...,
          6.0110353e-02,  3.2451293e-01, -2.0995583e-02],
        [ 7.1865958e-01, -4.8725206e-01,  5.1740402e-01, ...,
         -4.4012007e-01,  1.4553043e-01, -3.7544727e-02],
        [ 3.3223230e-01, -2.3270901e-01,  9.4876081e-02, ...,
         -2.5268167e-01,  3.2172003e-01,  8.1142318e-04],
        [ 1.2523208e+00,  3.5754293e-01,

### Tokenizers(分词器)

分词器是 NLP 管道的核心组件之一。它的目的只有一个：**将文本转换为模型可以处理的数据**——模型只能处理数字，因此分词器需要将我们的文本输入转换为数字数据。


#### 分词器的分类

在了解分词器原理之前，我们先来理解一下有分词器实现有哪些方式

 #### 基于单词的
基于单词的分词通常很容易设置和使用，只需一些规则，并且通常会产生不错的结果。
<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/word_based_tokenization.svg"/>

我们可以通过应用 Python 的 `split()`函数，使用空格将文本标记为单词.
```
tokenized_text = "Jim Henson was a puppeteer".split()
print(tokenized_text)
```
```
['Jim', 'Henson', 'was', 'a', 'puppeteer']

```

使用这种标记器，我们最终可以获得一些相当大的“词汇表”，其中词汇表是由我们语料库中拥有的独立标记的总数定义的。

每个单词都会分配一个 ID，从 0 开始，一直到词汇表的大小。该模型使用这些 ID 来识别每个单词。

如果我们想用基于单词的分词器完全覆盖一种语言，我们需要为该语言中的每个单词都有一个标识符，这将生成大量的标记。例如，英语中有超过 500,000 个单词，因此要构建从每个单词到输入 ID 的映射，我们需要跟踪这么多 ID。但是，像“dog”这样的词与“dogs”这样的词的表示方式不同，模型最初无法知道“dog”和“dogs”是相似的：它会将这两个词识别为不相关。

我们需要一个自定义标记来表示不在我们词汇表中的单词。这被称为“未知”令牌，通常表示为“ [unk]”或“”。如果您看到标记生成器正在生成大量此类标记，这通常是因为它无法检索单词的合理表示，并且您会在此过程中丢失信息。

**减少未知标记数量的一种方法是使用基于字符的分词器。**

 #### 基于字符

 基于字符的分词器将文本拆分为字符有两个主要好处：
 - 词汇量要小得多。
 - 词汇表外（未知）标记要少得多，因为每个单词都可以由字符构建。

 <img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/character_based_tokenization.svg"/>

 这种方法也并不完美。直观上来说，它的意义不大：每个字符本身并没有多大意义，而单词就是这样。然而，这又根据语言的不同而有所不同；例如，在中文中，每个字符比拉丁语言中的字符携带更多信息。
 还有，我们的模型最终会处理大量的标记：虽然一个单词在基于单词的标记生成器中只是一个标记，但它可以很容易地变成 10 个或更多标记当转换成字符时。

 为了充分利用这两种方法，我们可以使用结合了两种方法的第三种技术：**子词分词。**

 #### 基于子词分词

 子词分词算法依赖于这样的原则：频繁使用的词不应该被分割成更小的子词，但罕见的词应该被分解成有意义的子词。

 下面示例，显示子词分词算法如何标记序列"Let’s do tokenization!"

 <img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/bpe_subword.svg"/>

 ##### 其他
 - Byte-level BPE, as used in GPT-2
 - WordPiece, as used in BERT
 - SentencePiece or Unigram, as used in several multilingual models


#### 分词器加载和保存

加载和保存分词器与模型一样简单。它基于相同的两个方法：from_pretrained() 和 save_pretrained()。

加载使用与 BERT 相同的检查点训练的 BERT 分词器的方式与加载模型相同，
方式一：我们使用 BertTokenizer 类：

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

方式二：使用AutoTokenizer
与 AutoModel 类似， AutoTokenizer 类将根据检查点名称在库中获取正确的分词器类，并且可以直接与任何检查点一起使用：

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [9]:
tokenizer("Using a Transformer network is simple")

{'input_ids': [101, 7993, 170, 13809, 23763, 2443, 1110, 3014, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

#### encoding(编码)

将文本转换为数字称为编码。编码分两步过程完成：标记化，然后转换为输入 ID。

##### Tokenization （分词过程）

分词过程由分词器的 tokenize() 方法完成。结果就是得到token数量

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

sequence = "Using a Transformer network is simple"
tokens = tokenizer.tokenize(sequence)

print(tokens)

['Using', 'a', 'Trans', '##former', 'network', 'is', 'simple']


##### 从token到imput ID

到输入 ID 的转换由 tokenizer.convert_tokens_to_ids()  方法处理：

In [11]:
ids = tokenizer.convert_tokens_to_ids(tokens)

print(ids)

[7993, 170, 13809, 23763, 2443, 1110, 3014]


#### decoding(解码)

从词汇索引中，得到我们想要的字符串。可以使用 decode() 方法。

decode 方法不仅将索引转换回token，而且还将属于相同单词的token组合在一起以生成可读的句子。

In [12]:
decoded_string = tokenizer.decode([7993, 170, 11303, 1200, 2443, 1110, 3014])
print(decoded_string)

Using a transformer network is simple


### 处理多个序列

自己分部实现一次处理多条文本推理的情况，需要注意以下几点：

- 向模型传递文本分词后的数字列表时，需要在其上添加了一个维度

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."



Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


使用封装好的api如下，可以看到分词器不仅将输入 ID 列表转换为张量，还在其上添加了一个维度

In [19]:
tokenized_inputs = tokenizer(sequence, return_tensors="pt")
print(tokenized_inputs["input_ids"])

outputs = model(**tokenized_inputs)
print(outputs)

tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102]])
SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


所以我们在自己处理的时候，需要给ids增加一个维度，`torch.tensor([ids])`

In [17]:
tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
# 注意，这里给的是数组
input_ids = torch.tensor([ids])
print("Input IDs:", input_ids)

output = model(input_ids)
print("Logits:", output.logits)

Input IDs: tensor([[ 1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,  2607,
          2026,  2878,  2166,  1012]])
Logits: tensor([[-2.7276,  2.8789]], grad_fn=<AddmmBackward0>)


- 当使用多个文本推理的时候，每个文本长度可能不同，这个时候通常会填充输入。
填充通过向具有较少值的句子添加一个称为`padding token`的特殊单词来确保所有句子具有相同的长度。

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

sequence1_ids = [[200, 200, 200]]
sequence2_ids = [[200, 200]]
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

print(model(torch.tensor(sequence1_ids)).logits)
print(model(torch.tensor(sequence2_ids)).logits)
print(model(torch.tensor(batched_ids)).logits)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[ 1.5694, -1.3895]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)
tensor([[ 1.5694, -1.3895],
        [ 1.3374, -1.2163]], grad_fn=<AddmmBackward0>)


#### Attention masks（注意力忽略）

Attention masks 是与输入 ID 张量维度完全相同的张量，填充有 0 和 1。1 表示应注意相应的标记，0 表示不应注意相应的标记（即，应忽略它们）。

上面例子中，批处理的第二行应该与第二句话的逻辑相同，但我们得到了完全不同的值！就是因为Transformer 模型的关键特征是将每个标记置于上下文中的注意力层，这些将考虑填充标记。

为了在通过模型传递不同长度的单个句子或传递具有相同句子并应用填充的批次时获得相同的结果，我们需要告诉这些注意层忽略填充标记。这是通过使用注意力掩模来完成的。

In [21]:
batched_ids = [
    [200, 200, 200],
    [200, 200, tokenizer.pad_token_id],
]

attention_mask = [
    [1, 1, 1],
    [1, 1, 0],
]

outputs = model(torch.tensor(batched_ids), attention_mask=torch.tensor(attention_mask))
print(outputs.logits)

tensor([[ 1.5694, -1.3895],
        [ 0.5803, -0.4125]], grad_fn=<AddmmBackward0>)


#### 长文本

对于 Transformer 模型，我们可以传递模型的序列长度是有限的。大多数模型可处理最多 512 或 1024 个标记的序列，并且当要求处理更长的序列时会崩溃。这个问题有两种解决方案：

- 使用支持的序列长度更长的模型。

- 文本截断

如何要处理很长的文本，可以查看这个两个模型：
[Longformer](https://huggingface.co/docs/transformers/model_doc/longformer) 和 [LED](https://huggingface.co/docs/transformers/model_doc/led)

如果选择截断的方式，可以使用 max_sequence_length：
```
sequence = sequence[:max_sequence_length]
```

### tokenizer 整合

上面我们探索了分词器的工作原理，并研究了分词、转换为输入 ID、填充、截断和注意力掩码。

这些都可以通过tokenizer 直接处理

In [22]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)

#### 处理多个序列

In [24]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

model_inputs = tokenizer(sequences)

#### 填充

In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

#### 截断序列

In [ ]:
equences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Will truncate the sequences that are longer than the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, truncation=True)

# Will truncate the sequences that are longer than the specified max length
model_inputs = tokenizer(sequences, max_length=8, truncation=True)

#### 处理到特定框架张量的转换

 "pt" 返回 PyTorch 张量，
"tf" 返回 TensorFlow 张量，
"np" 返回 NumPy 数组：

In [ ]:
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

# Returns PyTorch tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="pt")

# Returns TensorFlow tensors
model_inputs = tokenizer(sequences, padding=True, return_tensors="tf")

# Returns NumPy arrays
model_inputs = tokenizer(sequences, padding=True, return_tensors="np")

#### 特殊 tokens

In [25]:
sequence = "I've been waiting for a HuggingFace course my whole life."

model_inputs = tokenizer(sequence)
print(model_inputs["input_ids"])

tokens = tokenizer.tokenize(sequence)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[101, 1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012, 102]
[1045, 1005, 2310, 2042, 3403, 2005, 1037, 17662, 12172, 2607, 2026, 2878, 2166, 1012]


In [26]:
print(tokenizer.decode(model_inputs["input_ids"]))
print(tokenizer.decode(ids))

[CLS] i've been waiting for a huggingface course my whole life. [SEP]
i've been waiting for a huggingface course my whole life.


分词器在开头添加特殊单词 [CLS] ，在末尾添加特殊单词 [SEP] 。这是因为模型是用这些进行预训练的，因此为了获得相同的推理结果，我们还需要添加它们。

### transformers 使用总结

```python
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = ["I've been waiting for a HuggingFace course my whole life.", "So have I!"]

tokens = tokenizer(sequences, padding=True, truncation=True, return_tensors="pt")
output = model(**tokens)
```